In [43]:
import pandas as pd
import yfinance as yf
import numpy as np

In [25]:
tickers = ['DIA','MMM', 'AXP', 'AAPL', 'BA', 'CAT', 'CVX', 'CSCO', 'KO', 'DIS', 'XOM', 'GS', 'HD', 'IBM', 'INTC', 'JNJ', 'JPM', 'MCD', 'MRK', 'MSFT', 'NKE', 'PFE', 'PG', 'TRV', 'UNH', 'VZ', 'V', 'WMT', 'WBA']

## What type of day

In [54]:
start_date = '2019-05-07'
end_date = '2019-05-08'

In [33]:
data_open = yf.download(tickers, start_date, end_date, progress=False)['Open']
data_close = yf.download(tickers, start_date, end_date, progress=False)['Adj Close']
data_final = (data_close - data_open)/data_open
data_final

Ticker,AAPL,AXP,BA,CAT,CSCO,CVX,DIA,DIS,GS,HD,...,NKE,PFE,PG,TRV,UNH,V,VZ,WBA,WMT,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2019-05-07,-0.051423,-0.066007,-0.047681,-0.117814,-0.141886,-0.191496,-0.102162,-0.029179,-0.117838,-0.123501,...,-0.056196,-0.190283,-0.114051,-0.115435,-0.076539,-0.038085,-0.221129,-0.215059,-0.082475,-0.22206


### Must be a down day as every stock in the index returned negative on all 4 testing dates.

# Strategy: receive betas of all stocks and short them at the beginning of the day.

In [40]:
start_history = '2010-01-01'
end_history = '2020-01-01'
data = yf.download(tickers, start_history, end_history, progress=False)['Adj Close'].pct_change().dropna()
betas = {}

In [48]:
djia_returns.cov(data['AAPL'])/data['AAPL'].var()

0.3019532945095695

In [50]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
x = np.array(data['AAPL']).reshape(-1, 1)
y = np.array(djia_returns).reshape(-1, 1)
model.fit(x, y)
model.coef_[0]

array([0.30195329])

In [53]:
djia_returns = data['DIA']

for stock in data:
    beta = djia_returns.cov(data[stock])/data[stock].var()
    betas[stock] = beta
    
dict(sorted(betas.items(), key=lambda item: item[1]))

{'WBA': 0.2699295909508379,
 'AAPL': 0.3019532945095695,
 'NKE': 0.343218222155046,
 'CSCO': 0.3525059017893206,
 'UNH': 0.3571333991880971,
 'INTC': 0.36298519445407285,
 'GS': 0.38102381209248687,
 'V': 0.3833658344938661,
 'CAT': 0.38549928275935513,
 'WMT': 0.38889799884801296,
 'BA': 0.4067790134873251,
 'MSFT': 0.415209295624494,
 'JPM': 0.42100857955390897,
 'MRK': 0.42248911773272513,
 'VZ': 0.43030508559546776,
 'AXP': 0.44400531593794296,
 'DIS': 0.4588362420926675,
 'CVX': 0.47163190733871113,
 'PFE': 0.47634117342870963,
 'HD': 0.4785198757885383,
 'IBM': 0.48947251560807914,
 'MCD': 0.49655012736578014,
 'PG': 0.5147546292296002,
 'KO': 0.5435577669819728,
 'TRV': 0.5453673191817942,
 'XOM': 0.5508386063525039,
 'MMM': 0.5666427452681849,
 'JNJ': 0.6009728282072597,
 'DIA': 1.0000000000000004}

## Finding how severe the down moves were in each stock / if there's an explainable trend

In [65]:
start_dates = ['2019-01-03', '2019-01-04', '2019-01-18', '2019-05-07']
end_dates = ['2019-01-04', '2019-01-05', '2019-01-19', '2019-05-08']

pct_changes = {}

for stock in tickers:
    returns = []
    for i in range(4):
        open_price = yf.download(stock, start_dates[i], end_dates[i], progress=False)['Open']
        close_price = yf.download(stock, start_dates[i], end_dates[i], progress=False)['Adj Close']
        pct_change = ((close_price - open_price)/open_price).iloc[0]
        returns.append(pct_change)
    pct_changes[stock] = returns

In [67]:
total_returns = {key: sum(values) for key, values in pct_changes.items()}

{k: v for k, v in sorted(total_returns.items(), key=lambda item: item[1])}

{'VZ': -0.922376100380321,
 'XOM': -0.9219047447037517,
 'MMM': -0.9163372237113012,
 'IBM': -0.8848468186573261,
 'WBA': -0.842004818626519,
 'CVX': -0.8210818125286335,
 'PFE': -0.767246641149304,
 'MRK': -0.5969661904301884,
 'KO': -0.5838440476014434,
 'CSCO': -0.5565092225644671,
 'JPM': -0.5486966573691345,
 'JNJ': -0.5384252484186282,
 'INTC': -0.5165067429011051,
 'HD': -0.4727481329047105,
 'PG': -0.4668699958341974,
 'GS': -0.45869925582892995,
 'CAT': -0.45705930027206787,
 'TRV': -0.4525610372793441,
 'MCD': -0.44479266741694357,
 'DIA': -0.3949428153850907,
 'WMT': -0.32491988526959437,
 'UNH': -0.3191290014192406,
 'AXP': -0.23982576307415743,
 'MSFT': -0.21400501789488244,
 'NKE': -0.20069710811728045,
 'AAPL': -0.1669767913725706,
 'V': -0.14164343150754893,
 'BA': -0.12456649866491527,
 'DIS': -0.08652751427388293}